In [1]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import math

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
# Group 2

group_name = [
    "A1", "A2", "A3", "A4", "A5", "A6", "A7",
    "B1", "B2", "B3", "B4", "B5",
    "C1", "C2", "C3", "C4", 
    "D1", "D2", "D3", "D4", "D5", "D6", 
    "E1", "E2", "E3", "E4", "E5"
]

group = [
    [4],[11, 12],[34],[40],[10],[42],[41]
    ,[6, 8, 13, 17, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47],[25]
    ,[1, 31],[9, 32],[3],[33]
    ,[2, 14, 22, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45],[23]
    ,[5],[16, 24],[19, 20, 21, 49, 50, 51],[28, 30, 36, 60],[59]
]

data_path = "./Refined_Data/Test_Data/test_complete.csv"
save_path = "./Refined_Data/Test_Data/test_refined2.csv"

# Group 3

# group_name = [
#     "A1", "A2", "A3", "A4", "A5", "A6", "A7",
#     "B1", "B2", "B3", "B4", "B5",
#     "C1", "C2", "C3", "C4", 
#     "D1", "D2", "D3", "D4", "D5", "D6", 
#     "E1", "E2", "E3", "E4", "E5", "E6"
# ]

# group = [
#     [4],[11, 12],[34],[40],[10],[42],[41]
#     ,[6, 8, 13, 17, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47],[25]
#     ,[1, 31],[9, 32],[3],[33]
#     ,[2, 14, 22, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45],[23]
#     ,[5],[16, 24],[19, 20, 21, 49, 50, 51],[28, 36, 60],[59], [30]
# ]

# data_path = "./Refined_Data/Test_Data/test_complete.csv"
# save_path = "./Refined_Data/Test_Data/test_refined3.csv"

In [3]:
def get_pow(series):
    return math.pow(series, 0.16)

In [4]:
data = pd.read_csv(data_path, parse_dates = ["date_time"])

data['energy_group'] = 0

for i in range(len(group)):
    data.loc[data["num"].isin(group[i]), "energy_group"] = str(i)

data['num'] = data['num'].apply(str)
data['Week'] = data['Week'].apply(str)
data['24Hour'] = data['24Hour'].apply(str)
data['holiday'] = data['holiday'].apply(str)
data['Weekend'] = data['Weekend'].apply(str)

data['perceived_temperature'] = 13.12 + 0.6215*data['C'] - 11.37*data['m/s'].apply(get_pow) + 0.3965*data['m/s'].apply(get_pow)*data['C']
data['discomfort_index'] = (1.8 * data['C']) - 0.55 * (1 - (data['wet'] * 0.01)) * (1.8 * data['C'] - 26) + 32

data.to_csv(save_path, mode='w')